In [ ]:
# usual imports
import os
import numpy as np
import matplotlib.pyplot as plt
#from rail.estimation.algos.bpz_version.utils import RAIL_BPZ_DIR
from rail.pipelines.estimation.inform_all import InformPipeline
from rail.core import common_params
from rail.utils import catalog_utils
import qp
import ceci

### Pick the relevant set of parameter names, etc¶

In [ ]:
catalog_tag = 'roman_rubin'

### Setting common parameters to all the stages in the estimation

In [ ]:
catalog_utils.apply_defaults(catalog_tag)

### Set up the estimate pipeline

In [ ]:
pipe = InformPipeline()

In [ ]:
pipe.inform_knn.config.update(leaf_size=14)

In [ ]:
pipe.inform_knn.config

### Specify input model for each stages, and the spec_input file (same as the input_file)

In [ ]:
if catalog_tag == 'hsc':
    data_dir = '/global/cfs/cdirs/lsst/groups/PZ/HSCDATA/SPECTROSCOPY'
elif catalog_tag == 'dc2':
    data_dir = '/global/cfs/cdirs/lsst/groups/PZ/PhotoZDC2/run2.2i_dr6_v2/'
elif catalog_tag in ['rubin', 'roman_rubin']:
    data_dir = '/sdf/data/rubin/shared/pz/data/training/'

In [ ]:
if catalog_tag == 'hsc':
    photo_input_file = os.path.join(data_dir,'dered_223501_sz_match_pdr3_dud_NONDET.hdf5')
    spec_input_file = photo_input_file
elif catalog_tag == 'dc2':
    photo_input_file = os.path.join(data_dir,"Run2.2i_dr6_dereddened_tract_4852.hdf5")
    spec_input_file = os.path.join(data_dir,"Run2.2i_dr6_dereddened_tract_4437_test_bright.hdf5")
elif catalog_tag in ['rubin', 'roman_rubin']:
    photo_input_file = os.path.join(data_dir,"roman_rubin_2023_v1.1.3_parquet_healpixel_maglim_25.5_truth_to_observed_train-100000.parquet")
    spec_input_file = os.path.join(data_dir,"roman_rubin_2023_v1.1.3_parquet_healpixel_maglim_25.5_truth_to_observed_train-100000.parquet")

output_dir = os.path.join('projects', 'catalog_tag')
input_dict = pipe.default_input_dict.copy()
input_dict.update(
    input=photo_input_file,
)

In [ ]:
pipe_info = pipe.initialize(input_dict, dict(output_dir=output_dir, log_dir='.', resume=True), None)

In [ ]:
pipe.print_stages()

In [ ]:
pipe.save('inform_all.yml')

[For NERSC users!]

This won't work on jupyter.nersc.gov, for a test run, you need to

Add "name: local" to the "site" section in the 'inform_all.yml'
ssh into perlmutter.nersc.gov, set up the rail environment, and run "ceci estimate_all.yml"

In [ ]:
import ceci
pr = ceci.Pipeline.read('inform_all.yml')

In [ ]:
pr.run()

In [ ]:
# curl -O https://portal.nersc.gov/cfs/lsst/PZ/dered_223501_sz_match_pdr3_dud_NONDET.hdf5

In [ ]:
# OMP_NUM_THREADS=1   python3 -m ceci rail.estimation.algos.flexzboost.FlexZBoostInformer   --input=dered_223501_sz_match_pdr3_dud_NONDET.hdf5   --name=inform_fzboost   --config=tmp_inform_all_config.yml   --model=./model/estimator/model_FZBoost.hdf5



